## Install packages

- `kenlm`
- `datasets`
- `pycorrector`

```shell
pip install datasets
pip install kenlm
pip install -U pycorrector
```

In [1]:
# Colab
!pip install -U pycorrector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycorrector: filename=pycorrector-1.0.0-py3-none-any.whl size=4373305 sha256=214db0d30eb1d3259bace544d30a96865d97a4d9735a9a4b7b52877855b3c97c
  Stored in directory: /root/.cache/pip/wheels/1a/aa/44/026cd8cf5f66660558654b36abcb9216e87349ab7cfd6499a8
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184423 sha256=13bded00d9e040c3e7be3be9abbfd2f896bdcc300c65831c50fa3ffeb7502bab
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137

In [2]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


## Start to correct

In [5]:
from pycorrector import Corrector
import pycorrector

class EmailHelper(Corrector):

    # translate traditional Chinese to simplified Chinese
    def tra2sim(self, content):
        try:
            sim_content = pycorrector.traditional2simplified(content.replace("\n", '').replace(" ", ''))
            return [sim_content]
        except:
            sim_content = [pycorrector.traditional2simplified(tra_content.replace("\n", '').replace(" ", '')) for tra_content in content]
            return sim_content

    # get the values which the key equals to "target"
    def get_target(self, correct_dict):
        if len(correct_dict) == 1:
            self.error = correct_dict[0]["errors"]
            return correct_dict[0]["target"]
        else:
            self.error = [f"第{i+1}篇錯誤：{j['errors']}" for i, j in enumerate(correct_dict)]
            return [i["target"] for i in correct_dict]


    # translate traditional Chinese to simplified Chinese
    def sim2tra(self, content):
        if type(content) == str:
            tra_content = pycorrector.simplified2traditional(content)
        else:
            tra_content = [pycorrector.simplified2traditional(sim_content) for sim_content in content]
        return tra_content

In [10]:
emailhelper = EmailHelper()

### 單篇文章
> input: str <br>
> output: str

In [28]:
mail = """
    我寫這封信是為了向您於10/7星期午請一天假，

    因為我當天要參加一位置友的喪禮。

    我確認過公司的行事例了，

    我想我可以在那天之前把交辦事項處李好。

    感蟹您的諒解。
    """

In [29]:
mail = emailhelper.tra2sim(mail)
mail = emailhelper.correct_batch(mail)
mail = emailhelper.get_target(mail)
mail = emailhelper.sim2tra(mail)
mail

'我寫這封信是為了向您於10/7星期五請一天假，因為我當天要參加一位摯友的喪禮。我確認過公司的行事例了，我想我可以在那天之前把交辦事項處理好。感謝您的諒解。'

In [30]:
# 檢查文章之錯誤
emailhelper.error

[('午请', '五请', 17), ('置友', '挚友', 33), ('李', '理', 67), ('感蟹', '感谢', 70)]

### 多篇文章
> input: list <br>
> output: list

In [31]:
mail_list = [
    """
    我寫這封信是為了向您於10/7星期五請一天假，

    因為我當天要參加一位摯友的喪禮。

    我確認過公司的行事曆了，

    我想我可以在那天之前把交辦事項處理好。

    感謝您的諒解。
    """,
    """
    我寫這封信是為了向您於10/7星期午請一天假，

    因為我當天要參加一位置友的喪禮。

    我確認過公司的行事例了，

    我想我可以在那天之前把交辦事項處李好。

    感蟹您的諒解。
    """
]

In [32]:
mail_list = emailhelper.tra2sim(mail_list)
mail_list = emailhelper.correct_batch(mail_list)
mail_list = emailhelper.get_target(mail_list)
mail_list = emailhelper.sim2tra(mail_list)
mail_list

['我寫這封信是為了向您於10/7星期五請一天假，因為我當天要參加一位摯友的喪禮。我確認過公司的行事曆了，我想我可以在那天之前把交辦事項處理好。感謝您的諒解。',
 '我寫這封信是為了向您於10/7星期五請一天假，因為我當天要參加一位摯友的喪禮。我確認過公司的行事例了，我想我可以在那天之前把交辦事項處理好。感謝您的諒解。']

In [33]:
# 檢查文章之錯誤
for i in emailhelper.error:
    print(i)

第1篇錯誤：[]
第2篇錯誤：[('午请', '五请', 17), ('置友', '挚友', 33), ('李', '理', 67), ('感蟹', '感谢', 70)]
